# Music ETL Process Notebook

This code is the notebook version of the `processor.py` file which is the ETL script I use whenever I have a new batch of songs that need to processed, analyzed, and uploaded to my database.

Here I annotate each step of the process with explanations of what this code does.

In [9]:
#Imports
import sqlite3
import pandas as pd
import numpy as np
import matchering as mg
import json
from glob import glob
from tqdm import tqdm
from io import StringIO
import sys
import pathlib

import taglib
from datetime import datetime
import shutil
import os
from essentia.standard import MusicExtractor, YamlOutput,MetadataReader, PCA, YamlInput
import warnings
from zipfile import ZipFile
warnings.filterwarnings('ignore')
pd.set_option('max_colwidth', 100)

In [2]:
#Import utility functions from project_tools package
from project_tools.utils import json_opener, adapt_array, convert_array, tag_cleaner, digit2letters
from project_tools.models import Activator, Classifier

Since my sqlite database holds numpy arrays, which aren't a native data type, I need to register two functions `adapt_array` and `convert_array` which are used to handle the arrays.

In [3]:

sqlite3.register_adapter(np.ndarray, adapt_array)
sqlite3.register_converter("array", convert_array)

Connect to the `jaage.db` database

In [4]:
conn = sqlite3.connect("/Users/georgemcintire/projects/djing/jaage.db", detect_types= sqlite3.PARSE_DECLTYPES)
cur = conn.cursor()

`Loading Dock` is the directory in my hard drive which is the staging area for newly downloaded songs.

`DJ Hub` is where the original files are sent to after being processed.

In [54]:
load_path = "/Volumes/LaCie/Loading Dock/"
dj_hub  = "/Volumes/LaCie/DJ Hub/"

Sometimes downloaded songs are packaged as zip files, so if that is the case I unzip them.

In [55]:
zip_files = glob(load_path+"*.zip")
zip_files

[]

In [56]:
if len(zip_files) > 0:
    for z in zip_files:
        zf = ZipFile(z)
        zf.extractall(path=load_path)
        shutil.move(z,dj_hub)

Collect all the music files.

In [57]:
loading_files = pathlib.Path(load_path).glob("*[.wav, .mp3, .aiff]")

In [58]:
len_loading_files = len(list(loading_files))
print("There are {} files for the ETL pipeline".format(len_loading_files))

There are 1 files for the ETL pipeline


In [67]:
loading_files = pathlib.Path(load_path).glob("*[.wav, .mp3, .aiff]")

## Process Steps

### 1. Mastering


### 2. Essentia Features Extraction


### 3. Effnet Embeddings and Genre Activations

### 4. Style, Mood, and Genre Classification



### Mastering

Here I use the [matchering](https://github.com/sergree/matchering) package to adjust the sound and levels of the fresh batch of songs to make them more appropriate for DJing.


The tool works by taking the sound of an existing song and adjusts new songs on based on it.

In [68]:
#The song used to adjust other songs.
ref_file = '/Volumes/LaCie/DJ Hub/Rayko - Magnetized (Rayko rework).wav'
#The directory where all my music is stored.
collection = "Collection"

- Loop over the new files in the `loading_files` list, 

- Process them using matchering, 

- Then move them to collection

- Move original files to DJ Hub

In [69]:
new_file_paths = []
for f in tqdm(loading_files):
    out_stem = f.stem
    out_path = f.parent.parent/collection/f.stem
    out_path = out_path.as_posix() +".wav"
    
    mg.process(target= f.as_posix(),
              reference=ref_file, 
              results = [mg.pcm24(out_path)])
    
    load_tags = taglib.File(f.as_posix())
    mastered_tags = taglib.File(out_path)
    mastered_tags.tags = load_tags.tags
    mastered_tags.save()
    
    new_file_paths.append(out_path)
    
    try:
        shutil.move(f.as_posix(), dj_hub)
    except:
        print(f, "already exists")
        os.remove(f.as_posix())
    

1it [00:51, 51.83s/it]


**Before I move on to the next step I head over to RekordBox (the program I use to manage my library) and upload the new batch of songs on there and edit their metadata tags**

### Music Extraction

I use essentia to extract a variety of musical features and metadata from the songs.

The full description of what the `MusicExtractor` tool does can be found [here](https://essentia.upf.edu/tutorial_extractors_musicextractor.html) 

In [70]:
copied_paths = new_file_paths[:]
new_file_paths = []
for i in copied_paths:
    if os.path.exists(i):
        new_file_paths.append(i)
        
len(new_file_paths)

1

Initialize the extractor object

In [72]:
music_ext = MusicExtractor(lowlevelStats=['mean', 'stdev'],
                                    rhythmStats=['mean', 'stdev', "max", "min", "median"],
                                    tonalStats=['mean', 'stdev'],
                           mfccStats = ["mean", "cov"],
                           gfccStats = ["mean", "cov"])

Iterate the new batch, extract their data, and collect it in a list called `extracted_files`

In [73]:
out_dir = '../temp_features/'
extracted_files = []
id_2_paths = {}

for fil in tqdm(new_file_paths, total = len(new_file_paths)):
    try:
        features, _ = music_ext(fil)
        idd = features['metadata.audio_properties.md5_encoded']
        YamlOutput(filename= out_dir+"features.json", format="json")(features)
        json_data = json_opener(out_dir+"features.json")
        id_2_paths[idd] = fil
        extracted_files.append(json_data)
    except Exception as e:
        print(e)

  0%|                                                     | 0/1 [00:00<?, ?it/s][   INFO   ] MusicExtractor: Read metadata
[   INFO   ] MusicExtractor: Compute md5 audio hash, codec, length, and EBU 128 loudness
[   INFO   ] MusicExtractor: Replay gain
[   INFO   ] MusicExtractor: Compute audio features
[   INFO   ] MusicExtractor: Compute aggregation
[   INFO   ] All done
100%|█████████████████████████████████████████████| 1/1 [00:13<00:00, 13.41s/it]


Convert `extracted_files` to a pandas dataframe for easier data handling.

In [74]:
extracted = pd.json_normalize(extracted_files)
extracted.columns = extracted.columns.str.replace(".", "_")

Rename the `metadata_audio_properties_md5_encoded` column to `sid`. This property serves as the unique ids for my songs, `sid` is short for song id.

In [75]:
extracted.rename(columns={"metadata_audio_properties_md5_encoded":"sid"}, inplace=True)

Load in array of column names which are used to filter unnecessary data in the `extracted` dataframe.

In [76]:
drop_cols = np.load("../drop_cols.pkl", allow_pickle=True).tolist()
extracted.drop(drop_cols, axis = 1, inplace=True, 
               errors="ignore"
              )
extracted.set_index("sid", inplace=True)
extracted.shape

(1, 153)

Separate the metadata from the whole dataset, by creating a new dataframe called `meta_df`

In [77]:
cols = extracted.columns

meta_cols = cols[cols.str.startswith("meta")]
non_meta_cols = cols[~cols.str.startswith("meta")]

meta_df = extracted[meta_cols].copy()
extracted.drop(meta_cols, axis = 1, inplace=True)

This process right is used divide the remaining data in `extracted` into a dataframe where the values are lists and another where they are not.

In [78]:
list_cols = extracted.columns[extracted.iloc[0].apply(lambda x:type(x)) == list]
no_list_cols = extracted.columns[extracted.iloc[0].apply(lambda x:type(x)) != list]
list_data = extracted[list_cols]
no_list_data = extracted[no_list_cols]

`tag_cleaner` is used to deal with null values and empty lists in `meta_df`

In [79]:
meta_df = meta_df.applymap(tag_cleaner)

In [80]:
meta_df.columns = meta_df.columns.str.split("_").map(lambda x:x[-1])

In [81]:
meta_df.rename(columns={"name":"file_name"}, inplace=True)

The clean version of `meta_df`

In [82]:
meta_df.head()

,length,gain,codec,file_name,album,artist,date,title
sid,,,,,,,,
cd25d43df8d225f17346b27bf435ee11,306.025574,-13.738892,pcm_s24le,Turn Me On.wav,Move,Demuja,2020,Turn Me On


Before uploading `meta_df` to the `tags` table in the db, I need to make sure the column name are aligned. I import the columns from `tags` and use that to filter `meta_df`

In [83]:
tags_cols = pd.read_sql("SELECT * FROM tags LIMIT 1", con = conn).set_index('sid').columns.tolist()

In [84]:
meta_cols = [i for i in meta_df.columns if i in tags_cols]
meta_cols

['length', 'gain', 'codec', 'file_name', 'album', 'artist', 'date', 'title']

Append `meta_df` to the `tags` table in the db.

In [85]:
meta_df[meta_cols].to_sql("tags", con=conn, if_exists = "append")

Now it's time to update the `files` table which is how I connect the song paths to their unique ids.

In [86]:
files = pd.DataFrame(id_2_paths.items(), columns=["sid", "file_path"])

In [87]:
files.to_sql("files", con = conn, if_exists="append", index = False)

Divide the `no_list_data` dataframe into three sections: tonal, lowlevel, rhythm. These features are explained on the essentia website.

In [88]:
cols = no_list_data.columns
tonal_cols = cols[cols.str.startswith("tonal")]
lowlevel_cols = cols[cols.str.startswith("lowlevel")]
rhythm_cols = cols[cols.str.startswith("rhyt")]

tonal_df = no_list_data[tonal_cols]
lowlevel_df = no_list_data[lowlevel_cols]
rhythm_df = no_list_data[rhythm_cols]

Use those dataframes to update their corresponding tables in the db.

In [89]:
tonal_df.to_sql("tonal_features", con=conn, if_exists="append")
lowlevel_df.to_sql("lowlevel_features", con=conn, if_exists="append")
rhythm_df.to_sql("rhythm_features", con=conn, if_exists="append")

Upload the dataframes with list values to the db.

In [90]:
for col in tqdm(list_cols):
    ser = list_data[col].apply(pd.Series)
    ser.columns = col + "_"+ ser.columns.astype(str)
    ser.to_sql(col+"_tbl", con = conn,if_exists="append")

100%|███████████████████████████████████████████| 24/24 [00:00<00:00, 87.66it/s]


### EffNet Embbedings and Genre Classifications

I use the [discogs-effnet model](https://essentia.upf.edu/models.html#discogs-effnet) to generate Nx1280 embeddings and activation scores for 400 genres.

Those embeddings are then used later on as the input data for the other classification models.

The metadata on this model can be found [here](https://essentia.upf.edu/models/music-style-classification/discogs-effnet/discogs-effnet-bs64-1.json)

In [91]:
path2id = {v:k for k, v in id_2_paths.items()}

Initialize the `Activator` which is the tool I used to preprocess the raw song data, generate the predictions and then upload them to the database.

In [92]:
act = Activator(input_length=2.05, 
                model_path="../onnx_models/discogs-effnet-bsdynamic-1.onnx",
                   pathid_dict=path2id)

Grab the columns names from the `effnet_genres` table

In [93]:
gcols = pd.read_sql_query("SELECT * FROM effnet_genres LIMIT 1 ", con = conn).columns[1:].tolist()
# gcols[:5]

- Iterate over the new batch of songs

- Generate the activations and embeddings.

- Upload them to their corresponding tables in the database.

In [94]:
for song in act.batch_inference():
    with conn:
        sid, sf, output = song
        genre_acts = output["activations"]
        embeds = output["embeddings"]
        genre_acts = [np.expand_dims(genre_acts[:, i], 0) for i in range(400)]
        genre_acts = pd.DataFrame(index = [sid], data = [genre_acts], columns=gcols)
        cur.execute("INSERT INTO effnet_embeddings (sid, effnet_embedding) values (?,?)", 
                    (sid, np.expand_dims(embeds,0)))
        genre_acts.to_sql("effnet_genres", con=conn, if_exists="append", index=False)
    conn.commit()

100%|█████████████████████████████████████████████| 1/1 [00:02<00:00,  2.38s/it]


### Classification Head Models

This section is where I use the discogs-effnet embeddings to generate predictions from a variety of music [style and mood classification models](https://essentia.upf.edu/models.html#classification-heads).

These models classify attributes such as danceability, happiness, relaxedness, and more.


The `onnx_models` directory hosts all the downloaded models. The `json_info` subdirectory hosts all their corresponding metadata.

I collect all the models and their metadata here.

I have other models but for now I'm only working with the effnet ones.

In [95]:
model_paths = sorted(glob("../onnx_models/*.onnx"))
model_infos = sorted(glob("../onnx_models/json_info/*.json"))
effnet_models = [{"model": model_paths[i], 
                  "json":model_infos[i]} for i in range(len(model_paths)) if "effnet" in model_paths[i]]

In [96]:
effnet_models = effnet_models[:2] + effnet_models[4:]

In [97]:
new_ids = list(path2id.values())

- Iterate over all the models.

- Generate batch inferences from them. The models are feed data queried from the database.

- Upload the predictions to the database.

In [ ]:
for em in effnet_models:
    cls = Classifier(em, new_ids=new_ids)
    cls.batch_inference()
    cls.conn.commit()
    print("Completed => ", cls.table_name, "\n\n")

100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 375.80it/s]


Completed =>  approachability_2c_effnet_discogs_1_activations 




100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 210.44it/s]


Completed =>  danceability_effnet_discogs_1_activations 




100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 214.32it/s]


...

Completed =>  mood_aggressive_effnet_discogs_1_activations 




100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 333.46it/s]


Completed =>  mood_happy_effnet_discogs_1_activations 




100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 348.94it/s]


Completed =>  mood_party_effnet_discogs_1_activations 


